# Projet Final Apache Spark

**Nom Etudiant :**  SECK

**Prenom Etudiant:**  El Hadji Ahmadou

**Classe :**  Deuxieme annee cycle ingénieur (ING2)


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Lundi 28 Fevrier 2023 à 23h 59** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [ ]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

ModuleNotFoundError: No module named 'org'

Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [ ]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [ ]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
                        StructField("UnitID", StringType, true),
                        StructField("IncidentNumber", IntegerType, true),
                        StructField("CallType", StringType, true),                  
                        StructField("CallDate", StringType, true),      
                        StructField("WatchDate", StringType, true),
                        StructField("ReceivedDtTm", StringType, true),
                        StructField("EntryDtTm", StringType, true),
                        StructField("DispachDtTm", StringType, true),
                        StructField("ResponseDtTm", StringType, true),
                        StructField("OnSceneDtTm", StringType, true),
                        StructField("TransportDtTm", StringType, true),
                        StructField("HopitalDtTm", StringType, true),
                        StructField("CallFinalDisposition", StringType, true),
                        StructField("AvailableDtTm", StringType, true),
                        StructField("Address", StringType, true),       
                        StructField("City", StringType, true),       
                        StructField("Zipcode", IntegerType, true),       
                        StructField("Battalion", StringType, true),                 
                        StructField("StationArea", StringType, true),       
                        StructField("Box", StringType, true),       
                        StructField("OriginalPriority", StringType, true),       
                        StructField("Priority", StringType, true),       
                        StructField("FinalPriority", IntegerType, true),       
                        StructField("ALSUnit", BooleanType, true),       
                        StructField("CallTypeGroup", StringType, true),
                        StructField("NumAlarms", IntegerType, true),
                        StructField("UnitType", StringType, true),
                        StructField("UnitSequenceInCallDispatch", IntegerType, true),
                        StructField("FirePreventionDistrict", StringType, true),
                        StructField("SupervisorDistrict", StringType, true),
                        StructField("Neighborhood", StringType, true),
                        StructField("Location", StringType, true),
                        StructField("RowID", StringType, true),
                        StructField("Delay", FloatType, true)))

In [ ]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

Nous allons mettre en cache le Dataframe

In [ ]:
fireDF.cache()

In [ ]:
fireDF.count()

In [ ]:
fireDF.printSchema()

In [ ]:
fireDF.show(5)

Filtrage des d'appels de type "Medical Incident"

In [ ]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [ ]:
// Reponse 1
/*
fireDF.select("CallType").filter($"CallType" =!= "null").distinct().count()
*/

### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [ ]:
// Reponse 2
/*
fireDF.select("CallType").where($"CallTypeGroup" === "Fire").filter($"CallType" =!= "null").distinct().show()
*/

### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [ ]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
/*
newFireDF.select("CallNumber","ResponseDelayedinMins").where($"ResponseDelayedinMins"<5).show()
*/

### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [ ]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

### Question 4
**Quels sont les types d'appels les plus courants?**

In [ ]:
//Reponse 4
/*
val mostCommonCallTypes = fireTSDF.groupBy("CallType").count().orderBy($"count".desc).show()
*/

### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [ ]:
//Reponse 5-a
/*
fireTSDF.groupBy("CallType","Zipcode").count().orderBy($"count".desc).select("Zipcode","CallType","count").show()
*/

### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [ ]:
//Reponse 5-b
/*
fireTSDF.select("Neighborhood", "Zipcode").where($"Zipcode"===94102 || $"Zipcode"===94103).distinct().show()
*/

### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [ ]:
//Reponse 6
/*
fireTSDF.agg(count("*") as "Total"avg("ResponseDelayedinMins") as "moyenne",min("ResponseDelayedinMins") as "minimum",max("ResponseDelayedinMins") as "maximum").show()
*/

### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [ ]:
//Reponse 7-a
/*
fireTSDF.select(year($"IncidentDate").alias("year")).distinct().count()
*/

### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [ ]:
//Reponse 7-b
/*
fireTSDF.filter(year($"IncidentDate") === 2018).groupBy(weekofyear($"IncidentDate").alias("week")).count().orderBy(desc("count")).first().getInt(0)
*/

### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [ ]:
//Reponse 8
/*
fireTSDF.filter(year($"IncidentDate") === 2018 && $"Neighborhood"=!="None").groupBy($"Neighborhood").agg(avg($"ResponseDelayedinMins").alias("Moyenne_temps_reponse")).sort(desc("Moyenne_temps_reponse")).show()
*/

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [ ]:
//Reponse 9
/*
fireTSDF.write.parquet("C:/Users/sysal/Documents/Ecpi/Big data/apache-spark/Projet_big_data")
*/

### Question 10
**Comment relire les données stockée en format Parquet?**

In [ ]:
//Reponse 10
/*
val df = spark.read.parquet("C:/Users/sysal/Documents/Ecpi/Big data/apache-spark/Projet_big_data")
*/

## FIN DE PROJET